In [1]:
%load_ext autoreload
%autoreload 2
%autosave 0

Autosave disabled


In [2]:
from etl import DATADIR, ErgastF1

In [3]:
self = ErgastF1(DATADIR / 'ergast/f1.zip')
self

ErgastF1(path=/context/data/ergast/f1.zip)

In [4]:
from itertools import permutations
from pandas import DataFrame

f1 = self

cols = 'season race team source target'.split()
data = (
    self.results
    .join(f1.races['season round race'.split()], on='id_race')
    .join(f1.drivers['driver'], on='id_driver')
    .join(f1.teams['team'], on='id_team')
    .sort_values('season round driver'.split())
    .groupby('season race team'.split(), sort=False)
    ['driver']
)
data = ( (*k, *x) for k,v in data for x in permutations(v,2) )
data = (
    DataFrame(data, columns='season race team source target'.split())
    .drop_duplicates(subset='source target'.split())
    .reset_index(drop=True)
)
data

,season,race,team,source,target
0,1950,British Grand Prix,ERA,Bob Gerard,Cuth Harrison
1,1950,British Grand Prix,ERA,Bob Gerard,Leslie Johnson
2,1950,British Grand Prix,ERA,Bob Gerard,Peter Walker
3,1950,British Grand Prix,ERA,Bob Gerard,Tony Rolt
4,1950,British Grand Prix,ERA,Cuth Harrison,Bob Gerard
...,...,...,...,...,...
9983,2019,Australian Grand Prix,Red Bull,Pierre Gasly,Max Verstappen
9984,2019,Belgian Grand Prix,Red Bull,Alexander Albon,Max Verstappen
9985,2019,Belgian Grand Prix,Red Bull,Max Verstappen,Alexander Albon
9986,2019,Belgian Grand Prix,Toro Rosso,Daniil Kvyat,Pierre Gasly


In [9]:
from pandas import Series

core = 'Martin Brundle'
source = data['source']
target = data['target']
degree = Series(-1, index=source.unique())

mates = {core}
for i in range(10):
    degree[mates] = i
    mates = set(target[source.isin(mates)])
    mates -= set(degree[degree.ne(-1)].index)

print(f"{degree.eq(0).sum()} {core}")
print(f"{degree.eq(-1).sum()} drivers not connected")
print(f"{degree.gt(0).sum()} drivers within {degree.max()} degrees")

degree

1 Martin Brundle
11 drivers not connected
789 drivers within 7 degrees


Bob Gerard         4
Cuth Harrison      5
Leslie Johnson     5
Peter Walker       5
Tony Rolt          5
                  ..
Charles Leclerc    5
Sergey Sirotkin    4
Alexander Albon    5
Lando Norris       4
George Russell     5
Length: 801, dtype: int64

In [10]:
degree['Kimi Räikkönen']

3